# `mammos_entity` quickstart

## Some definitions

- a `mammos_units.Quantity` is an object that carries a value and units.

- a `mammos_entity.Entity` is a quantity, which in addition links its definition in the EMMO ontology and the [MaMMoS additions for magnetic materials](https://github.com/MaMMoS-project/MagneticMaterialsOntology)


In [1]:
import mammos_entity as me

## Creating entities

For entities that are important in MaMMoS, there are [convenient attributes](https://mammos-project.github.io/mammos/api/mammos_entity.html) to define those. For example for the saturation magnetization Ms:

In [2]:
Ms = me.Ms(800e3)  # defines Ms = 8e5 A/m  (default units are SI, i.e. A/m here)

In [3]:
Ms

SpontaneousMagnetization(value=800000.0, unit=A / m)

If no units are provided, then the ontology units are used, such as `A/m` in the example above. These units are SI units without numerical prefactors (such as kilo, milli, etc.)

If units are provided, these are compared with the units expected for that entity. An error is raised if they do not match. This is good practice for extra clarity.

In [4]:
M1 = me.Ms(800e3, "A/m")
M1

SpontaneousMagnetization(value=800000.0, unit=A / m)

Providing units can be useful, if numerical prefactors are used (such as kilo):

In [5]:
M2 = me.Ms(800, "kA/m")
M2

SpontaneousMagnetization(value=800.0, unit=kA / m)

In [6]:
M1 == M2

True

We can create an entity from a Quantity as well:

In [7]:
import mammos_units as u

Ms_quantity = 800e3 * u.A / u.m
Ms_quantity

<Quantity 800000. A / m>

In [8]:
me.Ms(Ms_quantity)

SpontaneousMagnetization(value=800000.0, unit=A / m)

If we like to express spontaneous magnetization in Tesla, we can convert it to the required A/m:

In [9]:
me.Ms(1.2, "T")  # Tesla not compatible with A/m

UnitConversionError: The unit 'T' is not equivalent to the unit of SpontaneousMagnetization 'A / m'

In [10]:
# enable implicit conversion from here on:
u.set_enabled_equivalencies(u.magnetic_flux_field())

Js = 1.2 * u.T  # Quantity in Tesla

me.Ms(Js.to("A/m"))  # Convert Quantity to A/m and create entity

SpontaneousMagnetization(value=954929.6580315315, unit=A / m)

## Access to ontology

Each entity object knows its label in the ontology:

In [11]:
Ms.ontology_label

'SpontaneousMagnetization'

The ontology attribute links to an [owlready2](https://github.com/pwin/owlready2) object created by the [EMMOntoPy](https://emmo-repo.github.io/EMMOntoPy/stable/) package.

When saving data to a file, this `ontology_label_with_iri` might be useful to save in the metadata as it returns a string containing the ontology label together with the unique identifier of the ontology entry (IRI is the Internationalized Resource Identifier).

In [12]:
Ms.ontology_label_with_iri

'SpontaneousMagnetization https://w3id.org/emmo/domain/magnetic_material#EMMO_032731f8-874d-5efb-9c9d-6dafaa17ef25'

We can use all attributes of the ontology object through `Ms.ontology`:

In [13]:
Ms.ontology.get_annotations()

{'prefLabel': [locstr('SpontaneousMagnetization', 'en')],
 'elucidation': [locstr('The spontaneous magnetization, Ms, of a ferromagnet is the result\nof alignment of the magnetic moments of individual atoms. Ms exists\nwithin a domain of a ferromagnet.', 'en')],
 'altLabel': ['Ms'],
 'wikipediaReference': ['https://en.wikipedia.org/wiki/Spontaneous_magnetization'],
 'IECEntry': ['https://www.electropedia.org/iev/iev.nsf/display?openform&ievref=221-02-41']}

In [14]:
Ms.ontology.get_class_properties()

{core.prefLabel,
 emmo.hasMeasurementUnit,
 magnetic_material_mammos.wikipediaReference,
 magnetic_material_mammos.IECEntry,
 emmo.elucidation,
 core.altLabel}

In [15]:
print(Ms.ontology.elucidation[0])

The spontaneous magnetization, Ms, of a ferromagnet is the result
of alignment of the magnetic moments of individual atoms. Ms exists
within a domain of a ferromagnet.


## Entities behaves like Quantity

Each Entity has all the attributes and methods that are available for Quantities. See [mammos_units examples](https://mammos-project.github.io/mammos/examples/mammos-units/index.html) for details.


In [16]:
Ms.value

800000.0

In [17]:
Ms.unit

Unit("A / m")

In [18]:
Ms.to("kA/m")

SpontaneousMagnetization(value=800.0, unit=kA / m)

Numerical operations with entities result in quantities (because `EMMOntoPy` does not support this):

In [19]:
Ms**2

<Quantity 6.4e+11 A2 / m2>

To convent an entity to a quantity, we can use the `quantity` attribute:

In [20]:
Ms.quantity

<Quantity 800000. A / m>

## Defining vector entities (Example Zeeman field)

In [21]:
H = me.H([1e4, 1e4, 1e4], "A/m")
H

ExternalMagneticField(value=[10000. 10000. 10000.], unit=A / m)

In [22]:
H.ontology

magnetic_material_mammos.ExternalMagneticField

In [23]:
print(H.ontology.elucidation[0])

The external field H′, acting on a sample that is produced by
electric currents or the stray field of magnets outside the sample
volume, is often called the applied field.


## Does `mammos_entity` not provide your preferred entity?

The list of convenience attributes is at 
https://mammos-project.github.io/mammos/api/mammos_entity.html

If the desired entity is not available, we can search the EMMO ontology (including the `magnetic_material_mammos` additions), for example for entity labels containing the string "Magnetization":

In [24]:
me.mammos_ontology.get_by_label_all("*Field*")

{emmo.ElectricDisplacementFieldUnit,
 emmo.ElectricFieldStrength,
 emmo.ElectricFieldStrengthUnit,
 emmo.MagneticFieldStrength,
 emmo.MagneticFieldStrengthUnit,
 magnetic_material_mammos.AnisotropyField,
 magnetic_material_mammos.DemagnetizingField,
 magnetic_material_mammos.ExternalMagneticField,
 magnetic_material_mammos.InternalMagneticField,
 magnetic_material_mammos.KneeField,
 magnetic_material_mammos.KneeFieldExternal,
 magnetic_material_mammos.SwitchingFieldCoercivity,
 magnetic_material_mammos.SwitchingFieldCoercivityExternal}

Once identified the right label, we create an entity like this:

In [25]:
me.Entity("ElectricFieldStrength", value=230)

ElectricFieldStrength(value=230.0, unit=kg m / (A s3))